In [1]:
!pip install llama-index-llms-openai
!pip install llama-index-extractors-entity

                                              0.0/15.4 MB ? eta -:--:--
     -                                        0.5/15.4 MB 15.2 MB/s eta 0:00:01
     -                                        0.7/15.4 MB 7.2 MB/s eta 0:00:03
     --                                       0.9/15.4 MB 6.1 MB/s eta 0:00:03
     --                                       1.0/15.4 MB 5.5 MB/s eta 0:00:03
     ---                                      1.2/15.4 MB 5.2 MB/s eta 0:00:03
     ---                                      1.4/15.4 MB 5.0 MB/s eta 0:00:03
     ----                                     1.6/15.4 MB 4.9 MB/s eta 0:00:03
     ----                                     1.8/15.4 MB 4.7 MB/s eta 0:00:03
     -----                                    2.0/15.4 MB 4.7 MB/s eta 0:00:03
     -----                                    2.2/15.4 MB 4.6 MB/s eta 0:00:03
     ------                                   2.3/15.4 MB 4.5 MB/s eta 0:00:03
     ------                                   2.5/15.4 MB 


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/50.9 kB ? eta -:--:--
     ---------------------------------------- 50.9/50.9 kB 2.5 MB/s eta 0:00:00
                                              0.0/198.6 MB ? eta -:--:--
                                             0.6/198.6 MB 12.2 MB/s eta 0:00:17
                                              0.8/198.6 MB 8.0 MB/s eta 0:00:25
                                              1.0/198.6 MB 6.0 MB/s eta 0:00:33
                                              1.1/198.6 MB 5.9 MB/s eta 0:00:34
                                              1.1/198.6 MB 5.9 MB/s eta 0:00:34
                                              1.1/198.6 MB 5.9 MB/s eta 0:00:34
                                              1.1/198.6 MB 3.6 MB/s eta 0:00:55
                                              1.1/198.6 MB 3.6 MB/s eta 0:00:55
                                              1.1/198.6 MB 3.6 MB/s eta 0:00:55
                                              2.2/198.6


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install llama-index


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/2.0 MB ? eta -:--:--
     ----------                               0.5/2.0 MB 17.2 MB/s eta 0:00:01
     --------------                           0.7/2.0 MB 9.3 MB/s eta 0:00:01
     ------------------                       0.9/2.0 MB 7.3 MB/s eta 0:00:01
     ----------------------                   1.1/2.0 MB 6.4 MB/s eta 0:00:01
     --------------------------               1.3/2.0 MB 5.9 MB/s eta 0:00:01
     -----------------------------            1.5/2.0 MB 5.5 MB/s eta 0:00:01
     ---------------------------------        1.7/2.0 MB 5.0 MB/s eta 0:00:01
     -------------------------------------    1.8/2.0 MB 4.9 MB/s eta 0:00:01
     ---------------------------------------  2.0/2.0 MB 4.8 MB/s eta 0:00:01
     ---------------------------------------- 2.0/2.0 MB 4.7 MB/s eta 0:00:00
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
                                              0.0/3.6 MB ? eta -:--:--
     -----           

In [2]:
import nest_asyncio

nest_asyncio.apply()

import os
import openai
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["OPENAI_API_KEY"] =  os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import MetadataMode

In [4]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo", max_tokens=512)

In [5]:
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    BaseExtractor,
)
from llama_index.extractors.entity import EntityExtractor
from llama_index.core.node_parser import TokenTextSplitter

text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)


class CustomExtractor(BaseExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": (
                    node.metadata["document_title"]
                    + "\n"
                    + node.metadata["excerpt_keywords"]
                )
            }
            for node in nodes
        ]
        return metadata_list


extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
    # EntityExtractor(prediction_threshold=0.5),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]

transformations = [text_splitter] + extractors

In [6]:
from llama_index.core import SimpleDirectoryReader

In [14]:
filenames = []
for root, dirs, files in os.walk("../data"):
    # cat = root.replace("../data\\", "")
    category = root.split("\\")[1:]
    # print(category)
    # print(root)
    for file in files:
        if len(category)==1:
            filenames.append(f'../data/{category[0]}/{file}')
        elif len(category)==2:
            filenames.append(f'../data/{category[0]}/{category[1]}/{file}')
        else:
            pass
        # print(os.path.join(root, file))

In [15]:
filenames

['../data/business_docs/Business Proposal.md',
 '../data/business_docs/Marketing Plan.md',
 '../data/business_docs/Progress Report.md',
 '../data/company_bylaws/About Instagram.md',
 '../data/company_bylaws/Board of Directors.md',
 '../data/company_bylaws/Diversity, Equity, and Inclusion.md',
 '../data/company_bylaws/Shareholders.md',
 '../data/financial_docs/Balance Sheet.md',
 '../data/financial_docs/Income Statement.md',
 '../data/financial_docs/Tax Return.md',
 '../data/hr_docs_filled/Employee Handbook.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_Ava Thomas.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_David Lee.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_Emily Brown.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_Ethan Rodriguez.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_Eva Kim.md',
 '../data/hr_docs_filled/employee_contracts/employee_contract_Jane Smith.md',
 '../data/hr_

In [16]:
# Note the uninformative document file name, which may be a common scenario in a production setting
instagram_docs = SimpleDirectoryReader(input_files=filenames).load_data()

Failed to load file ..\data\company_bylaws\About Instagram.md with error: 'utf-8' codec can't decode byte 0x97 in position 450: invalid start byte. Skipping...


In [17]:
instagram_docs

[Document(id_='be6349c7-a671-4c70-9a3b-9a06cb061c1a', embedding=None, metadata={'file_path': '..\\data\\business_docs\\Business Proposal.md', 'file_name': 'Business Proposal.md', 'file_size': 4602, 'creation_date': '2024-03-08', 'last_modified_date': '2024-03-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n\nCompany Name: Instagram\n\r\n---\r\n\r\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='cdd3b0f8-8beb-4db0-b789-164567d08b64', embedding=None, metadata={'file_path': '..\\data\\business_docs\\Business Proposal.md', 'file_name': 'Business Proposal.md', 'file_size': 4602, 'creation_date': '2024-03-08', 'last_modified_date': '2024

In [18]:
instagram_docs[0]

Document(id_='be6349c7-a671-4c70-9a3b-9a06cb061c1a', embedding=None, metadata={'file_path': '..\\data\\business_docs\\Business Proposal.md', 'file_name': 'Business Proposal.md', 'file_size': 4602, 'creation_date': '2024-03-08', 'last_modified_date': '2024-03-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='\n\nCompany Name: Instagram\n\r\n---\r\n\r\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [19]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

instagram_nodes = pipeline.run(documents=instagram_docs)

100%|██████████| 287/287 [01:26<00:00,  3.31it/s]


In [21]:
instagram_nodes[1].metadata

{'file_path': '..\\data\\business_docs\\Business Proposal.md',
 'file_name': 'Business Proposal.md',
 'file_size': 4602,
 'creation_date': '2024-03-08',
 'last_modified_date': '2024-03-08',
 'document_title': "Elevate Your Brand with Instagram's Social Media Marketing Services: A Comprehensive Guide",
 'questions_this_excerpt_can_answer': "1. What specific social media marketing services does Instagram offer to enhance brand presence and engagement?\n2. How does Instagram differentiate itself in the industry with its focus on visual content and community engagement?\n3. What are the terms and conditions outlined in Instagram's service agreement for their social media marketing services?",
 'excerpt_keywords': 'Instagram, social media marketing, brand presence, engagement, visual content, community engagement, digital marketers, audience reach, pricing structure, service agreement'}

In [22]:
from copy import deepcopy

nodes_no_metadata = deepcopy(instagram_nodes)
for node in nodes_no_metadata:
    node.metadata = {
        k: node.metadata[k]
        for k in node.metadata
        if k in ["page_label", "file_name"]
    }
print(
    "LLM sees:\n",
    (nodes_no_metadata)[9].get_content(metadata_mode=MetadataMode.LLM),
)

LLM sees:
 Board of Directors

The Board of Directors of Instagram consists of highly experienced professionals who provide strategic guidance and oversight to ensure the success and growth of the company. The board members bring diverse expertise and perspectives to the table, contributing to the overall vision and direction of Instagram.


In [23]:
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [24]:
index_no_metadata = VectorStoreIndex(
    nodes=nodes_no_metadata,
)
engine_no_metadata = index_no_metadata.as_query_engine(
    similarity_top_k=10, llm=OpenAI(model="gpt-3.5-turbo")
)

In [25]:
index = VectorStoreIndex(
    nodes=instagram_nodes
)
engine = index.as_query_engine(similarity_top_k=10, llm=OpenAI(model="gpt-3.5-turbo"))

In [26]:
from llama_index.core.question_gen import LLMQuestionGenerator
from llama_index.core.question_gen.prompts import (
    DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)


question_gen = LLMQuestionGenerator.from_defaults(
    llm=llm,
    prompt_template_str="""
        You are a helpful and informative bot that answers questions using text from the reference document included below. \
    Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
    However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
    strike a friendly and converstional tone. \
    Use your own knowledge base in addition to the information provided in the document to answer the question. \
    Make relevant assumptions and use your best judgement to answer the question. \
        """
    + DEFAULT_SUB_QUESTION_PROMPT_TMPL,
)

In [27]:
final_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name="instagram_business_documents",
                description="All the enterprise documents related to Instagram business",
            ),
        )
    ],
    question_gen=question_gen,
    use_async=True,
)

In [28]:
import time
start = time.time()
response = final_engine.query(
    """
    Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.
    Give your answer as a JSON.
    """
)
print(response.response)
end = time.time()
time_taken = end - start


Generated 3 sub questions.
[instagram_business_documents] Q: What is the progress report of the Feed Redesign project
[instagram_business_documents] Q: What is the progress report of the Stories Upgrade project
[instagram_business_documents] Q: What conclusion can be drawn from the progress reports of the Feed Redesign and Stories Upgrade projects
[instagram_business_documents] A: The Stories Upgrade project is completed.
[instagram_business_documents] A: The progress report of the Feed Redesign project indicates that the project is currently "In Progress." The achievements so far include the redesign of the layout and UI. However, the project is facing challenges related to technical issues. The action plans moving forward involve conducting bug fixes and testing. The resource allocation for this project includes 5 developers, and the financial updates show an allocation of $10,000.
[instagram_business_documents] A: The Feed Redesign project is currently in progress with achievements 

In [32]:
def generate_response(query):
    start = time.time()
    response = final_engine.query(f"""
        {query}
        Give your answer as a JSON.
        """
    )
    # print(response.response)
    end = time.time()
    time_taken = end - start
    return response.response, time_taken

In [30]:
questions = [
"What is Instagram's current business proposal?",
"What information does the progress report of Instagram contain?",
"What are the diversity and inclusion initiatives implemented by Instagram?",
"How do the terms of the employment contract boost employee retention?",
"What is the Marketing Objective for Influencer Collaboration Services?",
"What is the financial update for the Reels Optimization Project?",
"Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.",
"Who are the key personnel of Instagram?",
"How does Instagram promote community involvement?",
"What is the background of the CEO (Chief Executive Officer) of Instagram?",
"Who is the chairman of Instagram and which year did the person become the chairman of the board?",
"How has the Board of Directors contributed to Instagram's success?",
"Total number of work hours of Leo Kim?",
"What are the employee benefits provided to Ethan Rodriguez?",
"Name the employee working in the Security Department and what is his/her compensation?",
"What is the responsibility of the content moderator?",
"Which employee is getting paid $125,000 per year as compensation and which department does he work in?",
"Which employee did enter into the ESOP with Instagram on April 1, 2022?",
"How many number of shares were allocated to Liam Kim?",
"How much bonus does the Public Relations Manager receive?",
]

In [31]:
len(questions)

20

In [33]:
query = []
answers = []
total_time = []
for q in questions:
    query.append(q)
    answer, time_taken = generate_response(q)
    answers.append(answer)
    total_time.append(time_taken)

Generated 1 sub questions.
[instagram_business_documents] Q: What is Instagram's current business proposal
[instagram_business_documents] A: Instagram's current business proposal focuses on offering influencer collaboration services to connect brands with relevant influencers for impactful partnerships. The proposal includes services such as influencer discovery, outreach, negotiation, campaign management, and performance analysis. The pricing model for influencer collaborations is transparent and based on factors like influencer reach, engagement rates, content requirements, and campaign duration. Additionally, the terms and conditions of the influencer collaboration services agreement outline influencer selection criteria, content approvals, compensation details, and exclusivity clauses for mutual benefit and clarity.
Generated 1 sub questions.
[instagram_business_documents] Q: What is included in the progress report of Instagram
[instagram_business_documents] A: The progress report 

In [34]:
import pandas as pd
df = pd.DataFrame(list(zip(query, answers, total_time)), columns =['Query', 'Answer', 'Time Taken'])

In [35]:
df

,Query,Answer,Time Taken
0,What is Instagram's current business proposal?,"{\n ""Instagram's current business proposal"": ...",11.373541
1,What information does the progress report of I...,"{\n ""Information Included"": [\n ""Updates o...",6.280403
2,What are the diversity and inclusion initiativ...,"{\n ""Diversity Initiatives"": [\n ""St...",14.484236
3,How do the terms of the employment contract bo...,"{\n ""The terms of the employment contract c...",7.404701
4,What is the Marketing Objective for Influencer...,"{\n ""Marketing Objective"": ""The marketing obj...",4.995116
5,What is the financial update for the Reels Opt...,"{\n ""financial_update"": ""$7,000""\n}",3.727834
6,Compare the progress report of the Feed Redesi...,"{\n ""Feed Redesign Project"": {\n ""Status"":...",8.882985
7,Who are the key personnel of Instagram?,"{\n ""Key Personnel"": {\n ""Chairman o...",5.222167
8,How does Instagram promote community involvement?,"{\n ""Instagram promotes community involveme...",8.078805
9,What is the background of the CEO (Chief Execu...,"{\n ""background"": ""The CEO of Instagram has...",5.640261


In [36]:
df.to_csv("Llama_index_responses.csv", index=False)